In [1]:
import sys
import pandas as pd

In [2]:
df = pd.read_csv('Output.csv')

In [3]:
arr = df.columns

In [4]:
name_column = []
for i in df.columns:
    if("Name" in i):
        name_column.append(i)

In [5]:
final_columns = []
most_traded = ["NVDA", "AMZN", "AAPL", "TSLA", "MSFT", "META", "AMD", "INTC", "AVGO", "SMCI", "JPM", "MU"]
for i in most_traded:
    if(("Name_" + i) in name_column):
        final_columns.append("Name_" + i)
    

In [6]:
final_columns

['Name_NVDA',
 'Name_AMZN',
 'Name_AAPL',
 'Name_MSFT',
 'Name_AMD',
 'Name_INTC',
 'Name_AVGO',
 'Name_JPM',
 'Name_MU']

In [7]:
final_columns[:5]

['Name_NVDA', 'Name_AMZN', 'Name_AAPL', 'Name_MSFT', 'Name_AMD']

# Stacking For time series

In [8]:
import numpy as np
def create_stack(X, y, time_steps=50):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i:i + time_steps]
        Xs.append(v)
        ys.append(y.iloc[i + time_steps].close)
    return Xs, ys


In [9]:
df = df.astype(float)

In [10]:
df = df[["close", "open", "volume", "unix_time", ] + final_columns]

In [263]:
from tqdm import tqdm
X,y = [], []
count = 0
for i in tqdm(final_columns):
    df_slice = df[df[i] == 1]
    df_slice.reset_index(inplace = True)
    output = create_stack(df_slice,df_slice)
    for i in output[0]:
        X.append(i)
    for i in output[1]:
        y.append(i)
    if(count == 10):
        break
    count = count + 1
    

100%|██████████| 9/9 [00:01<00:00,  7.83it/s]


# Model Creation

In [264]:
assert(len(X) == len(y))

In [265]:
from sklearn.model_selection import train_test_split

In [266]:
X_train =  X[:9000]
y_train = y[:9000]
X_test = X[9000:]
y_test = y[9000:]

In [267]:
assert(len(X_train) == len(y_train))
assert(len(X_test) == len(y_test))
print("Length of train",len(X_train))
print("Length of test",len(X_test))

Length of train 9000
Length of test 1881


In [268]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

In [269]:
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

model = SVR(gamma=0.5)

# Training

In [237]:
model.fit(X_train.reshape(X_train.shape[0], -1), y_train)

SVR(gamma=0.5)

# Metrics

In [238]:
pred = model.predict(X_test.reshape(X_test.shape[0], -1))

In [239]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error


def metrics(Y_pred, Y_true):
    print("r2 score is : ",r2_score(Y_true,Y_pred))
    
    


In [240]:
metrics(pred, y_test)

r2 score is :  -0.23660720907726307


Since the data is not being normalized this result is hard to interpet without further analysis.

In [241]:
from statistics import mean 

mean(y)

119.51640197592133